# Set-Up

In [3]:
import numpy as np
import pandas as pd
import itertools
import matplotlib
import matplotlib.pyplot as plt
np.random.seed(123)
import scipy as sp
from scipy.interpolate import griddata
import scipy.interpolate as spi 
from scipy.stats import norm
import time
import matplotlib.ticker as ticker
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix

# Solving The Basic Model

The basic model involves a vertically-integrated monopolist. 

### Parameters

There are a few parameters we need to define. These will help us specify the equations below:

\begin{align}
&R_t = P_t(X_t)\cdot X_t = (a_1-a_2X_t) \cdot X_t\\
&C_{1,t} = c_1\cdot Y_t + (c_2+\eta_t)\cdot Y_t^2 + \mathbf{1}_{(Y_t>0)}\cdot c_f\\
&C_2 = s_1 \cdot N_t + s_2 \cdot N_t^2\\
&\eta_t = \rho \cdot \eta_{t-1} + \varepsilon_t
\end{align}

In [4]:
a_1 = 200 ## Intercept of demand curve
a_2 = 4 ## Slope of demand curve
c_1 = 0 ## linear coefficient for production cost equation
c_2 = 6 ## quadratic coefficient for production cost equation
c_f = 0 ## cost of placing an order
s_1 = 0.01 ## linear coefficient for cost of storing inventory
s_2 = 0.01 ## quadratic coefficient for cost of storing inventory
rho = 0.7 ## persistence parameter for cost, AR(1) process
beta = 0.99 ## The patience parameter

We also need to discretize the distribution of $\varepsilon$, the white noise process. 

In [5]:
def VFI(a_1,a_2,c_1,c_2,c_f,s_1,s_2,rho,beta, delta):

    epsilons, weights = np.polynomial.hermite.hermgauss(7)
    
    weights = weights/sum(weights)

    n = np.linspace(0,120,121) ## the grid of values n can take on
    eta = np.linspace(-8, 8, 7)

    omegas = pd.DataFrame(np.array(list(itertools.product(n, eta)))) ## Find the cartesian product of n, epsilons, to obtain states matrix

    omegas.columns = ('n', 'eta')
    
    print("Built State Space")

    x=np.linspace(0,50,51) ## grid of values for x
    y=np.linspace(0,100,101) ## grid of values for y

    choices=np.array(list(itertools.product(n,eta,y,x)))

    choices=choices[choices[:,3]<=choices[:,2]+choices[:,0]] 

    n_prime=(choices[:,2]-choices[:,3]+choices[:,0]) ## We add n_prime to the choices. This is implicitly chosen when x,y are chosen
    choices=np.append(choices,n_prime[:, np.newaxis],axis=1)

    cm = pd.DataFrame(choices)
    cm.columns = ('n', 'eta', 'y', 'x', 'n_p')
    
    print("Built Choice Space")

    p = np.maximum(a_1 - a_2*cm.x,0) ## It is further assumed price cannot fall below 0
    revenues = delta*p*cm.x 
    costs_1 = s_1*(cm.n)+s_2*(cm.n**2) # costs of holding inventory
    costs_2 = np.where(cm.y>0, c_1*cm.y+(c_2+cm.eta)*(cm.y**2) + c_f, c_1*cm.y+(c_2+cm.eta)*(cm.y**2)) # fixed cost only included if production exists
    flow_profits=revenues-costs_1-costs_2

    cm['flow_profits']=flow_profits
    
    print("Obtained Flow Profits")

    ## Construct the Various Scenarios
    for i in range(0,len(epsilons)):
        colname = 'scenario' + str(i)
        cm[colname] = rho*cm.eta+epsilons[i]

    for i in range(0,len(epsilons)): ## This is setup for the next part
        varname = 'l_n_eta_' + str(i)
        varname2 = 'u_n_eta_' + str(i)
        cm[varname]=eta[0]
        cm[varname2]=eta[-1]

    ate= eta[:: -1] ## The exact values for each scenario do not necessarily fall within our grid. We obtain the values by interpolation.
    for i in range(0,len(epsilons)):
        colname = 'scenario' + str(i)
        varname = 'l_n_eta_' + str(i)
        varname2 = 'u_n_eta_' + str(i)
        for j in eta:
            cm[varname] = np.where(cm[colname]>j, j, cm[varname]) ## finds the lower eta
        for j in ate:
            cm[varname2] = np.where(cm[colname]<j, j, cm[varname2]) ## find the upper eta 

    for i in range(0,len(epsilons)): ## This loop finds the appropriate "lower weight" to accord to the lower eta in the interpolation
        colname = 'l_n_eta_'+str(i)
        colname2 = 'u_n_eta_'+str(i)
        colname3 = 'scenario'+str(i)
        varname = 'w_l_'+str(i)
        cm[varname] = 0
        mask = (cm[colname]!=cm[colname2])
        cm.loc[mask, varname] = 1-((abs(cm.loc[mask, colname]- cm.loc[mask, colname3]))/abs((cm.loc[mask, colname2]-cm.loc[mask, colname])))

    for i in range(0,len(epsilons)): ## finds the upper weights
        varname = 'w_u_'+str(i)
        varname2 = 'w_l_'+str(i)
        cm[varname]= 1-cm[varname2]

    for i in range(0,len(epsilons)): ## Weighs upper, lower weight by the probability of the scenarios occuring. 
        varname= 'w_p_u_'+str(i)
        varname2 = 'w_p_l_'+str(i)
        colname = 'w_u_'+str(i)
        colname2 = 'w_l_'+str(i)
        cm[varname] = weights[i]*cm[colname] ## THIS NEEDS TO CHANGE. 
        cm[varname2] = weights[i]*cm[colname2] ## THIS NEEDS TO CHANGE. 

    ## this final operation collects our results into a simpler form. For each eta on our grid, a single weighted probability is given. 
    for i in range(0,len(eta)):
        colname = 'pr'+str(i)
        cm[colname]=0

    for i in range(0,len(eta)):
        colname = 'pr'+str(i)
        for j in range(0,len(epsilons)):
            varname = 'w_p_u_'+str(j)
            varname2 = 'u_n_eta_'+str(j)
            varname3 = 'w_p_l_'+str(j)
            varname4 = 'l_n_eta_'+str(j)
            cm[colname] = np.where(cm[varname2]==eta[i], cm[colname]+cm[varname], cm[colname])
            cm[colname] = np.where(cm[varname4]==eta[i], cm[colname]+cm[varname3], cm[colname])

    check1=cm.iloc[:,-7:] ## all should add to one, as for all states, one must go to some other state. 

    m=np.zeros((len(cm), (len(omegas))))

    for i in range(0,121,1): ## For each row, the column represent probabilities of transitioning to row from choice. 
        index = 7*i 
        index2 = index+7
        mask = (cm.n_p == i) ## Takes all the values in the data frame where those things are equal. 
        m[:,index:index2][mask] = check1.loc[mask]

    tran_matrix = csc_matrix(m)
    print('Transition Matrix Created')

    oldV=np.ones(len(omegas))

    print('Starting VFI')
    oldV=np.zeros(len(omegas))
    error=1
    i=0
    while error>0.1:
        s = time.time()
        cm['value']=flow_profits+beta*(tran_matrix @ oldV)
        e = time.time()
        newV=cm.groupby(['n','eta'])['value'].max()
        path=cm.loc[cm.groupby(['n', 'eta'])['value'].idxmax()]['flow_profits']
        error=max(abs(newV-oldV))
        oldV=newV
        print(error)
    return cm.loc[cm.groupby(['n', 'eta'])['value'].idxmax()]